# Predicting heart disease using machine learning

This notebook looks into using various Python-based machine learning and data science libraries in
an attempt to build machine learning model capable of predicting whether or not someone has heart
disease based on their medical attributes.

We're going to take the following approach:
1. Problem definition
2. Data
3. Evaluation
4. Features
5. Modelling
6. Experimentation

**1. Problem Definition**

In a statement,
> Given clinical parameters about a patient, can we predict whether or not they have heart disease?

**2. Data**

The original data came from Cleaveland database from the UCI Machine Learning Repository.
https://archive.ics.uci.edu/ml/datasets/Heart+Disease

There is also a version of it available on Kaggle. https://www.kaggle.com/ronitf/heart-disease-uci

**3. Evaluation**

> If we can reach 95% accuracy at predicting whether or not a patient has heart disease during the proof of concept, we'll pursue the project.

**4. Features**
This is where you'll get different information about each of the features in your data.

Data Dictionary:

* age: age in years
* sex: (1 = male; 0 = female)
* cp: chest pain type
    0: Typical angina: chest pain related decrease blood supply to the sugar
    1: Atypical angina: chest pain not related to heart
    2: Non-anginal pain: typically esophaegal spasma (non heart related)
    3: Asymptomatic: chest pain not showing signs of disease
* trestbps: resting blood pressure (in mm Hg on admission to the hospital) anything above 130-140 is             typically clause for concern
* chol: serum cholestoral in mg/dl
            * serum = LDL + HDL + .2 * triglycerides
            * above 200 is cause for concern
* fbs: (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)
            * '>126' mg/dl signals diabetes
* restecg: resting electrocardiographic results (values 0,1,2)
            * 0: Nothing to note
            * 1: ST-T Wave Abnormality
            * 2: Possible or definite left ventricular hypertrophy
* thalach: maximum heart rate achieved
* exang: exercise induced angina (1 = yes; 0 = no)
* oldpeak: ST depression induced by exercise relative to rest
* slope: the slope of the peak exercise ST segment
            * Value 1: upsloping
            * Value 2: flat
            * Value 3: downsloping
* ca: number of major vessels (0-3) colored by flourosopy
* thal: 3 = normal; 6 = fixed defect; 7 = reversable defect
* target: 1 or 0

## Preparing the tools

We're going to use Pandas, matplotlib and Numpy for data anlaysis and manipulations.

In [ ]:
# Import all the tools we need

# Regular EDA(Exploratory Data Analysis) and plotting libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# we want our plots to appear inside the notebook
%matplotlib inline 

# Models from scikit-learn for Classification
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Model Evaluations
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve

**Load data**

In [ ]:
df = pd.read_csv("heart-disease.csv")
df

## Data Exploration (EDA)

The goal here is to find out more about the data and become a subject matter expert on the dataset
you are working with.

1. What questions are you trying to solve?
2. What kind of data do we have and how do we treat different types?
3. What's missing from the data and how do you deal with it?
4. Where are the outliers and why should you care about them?
5. How can you add, change or remove features to get more out of your data?

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.target.value_counts()

In [ ]:
df.target.value_counts().plot.bar(color=["salmon", "lightblue"]);

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.describe()

### Heart Disease Frequency according to sex

In [ ]:
df.sex.value_counts()

In [ ]:
# Compare target with sex
pd.crosstab(df.target, df.sex)

In [ ]:
# Create plot of crosstab
pd.crosstab(df.target, df.sex).plot.bar(figsize=(10,6),
                                        color=["salmon","lightblue"]);
plt.title("Heart Disease frequency as per sex")
plt.xlabel("0: No Disease, 1: Disease")
plt.ylabel("Frequency")
plt.legend(["Female","Male"]);
plt.xticks(rotation=0);

### Age vs. Max Heart Rate for Heart Disease

In [ ]:
# Create another figure
plt.figure(figsize=(10,6))

# Scatter with positive examples
plt.scatter(df.age[df.target==1],
            df.thalach[df.target==1],
            c="salmon");

# Scatter with negative examples
plt.scatter(df.age[df.target==0],
            df.thalach[df.target==0],
            c="darkblue")

# Add some helpful info
plt.title("Heart Disease in function of Age and Max Heart Rate")
plt.xlabel("Age")
plt.ylabel("Max Heart Rate")
plt.legend(["Disease","No Disease"]);

In [ ]:
# Check the distribution of the age column with a histogram
df.age.plot.hist(color="red", rwidth=0.8);

### Heart Disease Frequency per Chest Pain Type

3. cp- chest pain type

    * 0: Typical angina: chest pain related decrease blood supply to the sugar.
    * 1: Atypical angina: chest pain not related to heart.
    * 2: Non-anginal pain: typically esophaegal spasma (non heart related)
    * 3: Asymptomatic: chest pain not showing signs of disease

In [ ]:
pd.crosstab(df.cp, df.target)

In [ ]:
# Make the crosstab more visual
pd.crosstab(df.cp,df.target).plot(kind="bar",
                                  figsize=(10,6),
                                  color=["red","darkblue"])

# Add some communication
plt.title("Heart Disease Frequency per Chest Pain Type")
plt.xlabel("Chest Pain Type")
plt.ylabel("Amount")
plt.legend(["No Disease","Disease"])
plt.xticks(rotation=0);

In [ ]:
# Building a Correlation matrix
df.corr()

In [ ]:
# Let's make our correlation matrix a little prettier
corr_matrix = df.corr()
fig,ax = plt.subplots(figsize=(15,10))
ax = sns.heatmap(corr_matrix,
                 annot=True,
                 linewidth=0.5,
                 fmt=".2f",
                 color=["Yellow","Green","Blue"]);
# bottom, top = ax.get_ylim()
# ax.set_ylim(bottom+0.5, top-0.5);

## 5. Modelling

In [ ]:
df.head()

In [ ]:
# Split the data into X and y
X = df.drop("target",axis = 1)
y = df.target

In [ ]:
X

In [ ]:
y

In [ ]:
# Train & Test Split
np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
X_train

In [ ]:
y_train, len(y_train)

Now we've got our data split into training & test sets, it's time to build a machine learning model.

We'll train it (find the patterns on the training set)

And we'll test it (use the patterns on the test set)

We're going to use 3 different machine learning models:
1. Logistic Regression
2. K-Nearest Neighbors Classifier
3. Random Forest Classifier

In [ ]:
# Put models in a dictionary
models = {"Logistic Regression": LogisticRegression(),
          "KNN": KNeighborsClassifier(),
          "Random Forest": RandomForestClassifier()}

# Create a function to fit and score models
def fit_and_score(models, X_train, X_test, y_train, y_test):
    """
    Fits and evaluates given machine learning models.
    models : a dict of different scikit-learn machine learning models
    X_train : training data (no labels)
    X_test : testing data (no labels)
    y_train : training labels
    y_test : test labels
    """
    # Set randm seed
    np.random.seed(42)
    # Make a dictionary to keep model scores
    model_scores={}
    # Loop through models
    for name,model in models.items():
        # Fit the model to the data
        model.fit(X_train,y_train)
        # Evaluate the model and append its score to model_scores
        model_scores[name]=model.score(X_test,y_test)
    return model_scores

In [ ]:
model_scores = fit_and_score(models=models,
                             X_train=X_train,
                             X_test=X_test,
                             y_train=y_train,          
                             y_test=y_test)
model_scores

**Model Comparison**

In [ ]:
model_compare = pd.DataFrame(model_scores, index = ["accuracy"])
model_compare

In [ ]:
model_compare.plot.bar()

In [ ]:
model_compare.T.plot.bar()

Now we've got a baseline model... and we know a model's first predictions aren't always what we should based our next steps off. What should we do?

Let's look at the following:
* Hyperparameter Tuning
* Feature importance
* Confusion Matrix
* Cross-validation
* Precision
* Recall
* F1-score
* Classification report
* ROC Curve
* Area under the curve (AUC)

#### HyperParameter Tuning (by hand)

In [ ]:
# Let's tune KNN

train_scores=[]
test_scores=[]

# Create a list of different values for n_neighbors
neighbors = range(1,21)

# Setup KNN instance
knn = KNeighborsClassifier()

# Loop through different n_Neighbors
for i in neighbors:
    knn.set_params(n_neighbors=i)
    
    # Fit the algorithm
    knn.fit(X_train, y_train)
    
    # Update the training scores list
    train_scores.append(knn.score(X_train, y_train))
    
    # Update the test scores list
    test_scores.append(knn.score(X_test, y_test))

In [ ]:
train_scores

In [ ]:
test_scores

In [ ]:
plt.plot(neighbors, train_scores, label = "Train score")
plt.plot(neighbors, test_scores, label = "Test score")
plt.xticks(np.arange(1,21,1))
plt.xlabel("Number of neighbors")
plt.ylabel("Model Scores")
plt.legend()

print(f"Maximum KNN Score on the test data: {max(test_scores)*100:.2f}%")

## HyperParameter Tuning by RandomizedSearchCV

We're going to tune:
* Logistic Regression
* Random Forest Classifier

...using RandomizedSearchCV

In [ ]:
# create a hyperparametr grid for LogisticRegression
log_reg_grid = {"C": np.logspace(-4,4,20),
                "solver": ["liblinear"]}

# Create a hyperparameter grid for RandomForestClassifer
rf_grid = {"n_estimators": np.arange(10,1000,50),
           "max_depth": [None,3,5,10],
           "min_samples_split": np.arange(2,20,2),
           "min_samples_leaf": np.arange(1,20,2)}

Now we've got hyperparameter grids setup for each of our models, let's tune them using RandomizedSearchCV

In [ ]:
# Tune Logistic Regression

np.random.seed(42)

# Setup Random hyperparameter search for Logistic Regression
rs_log_reg = RandomizedSearchCV(LogisticRegression(),
                                param_distributions=log_reg_grid,
                                cv=5,
                                n_iter=20,
                                verbose=True)
# Fit Random Hyperparameter search model for Logistic Regression
rs_log_reg.fit(X_train, y_train)

In [ ]:
rs_log_reg.best_params_

In [ ]:
rs_log_reg.score(X_test,y_test)

**Random Forest Classifier**

In [ ]:
# Tune RandomForestClassifier

np.random.seed(42)

# Setup Random hyperparameter search for RandomForestClassifier
rs_rf = RandomizedSearchCV(RandomForestClassifier(),
                           param_distributions=rf_grid,
                           cv=5,
                           n_iter=20,
                           verbose=True)
# Fit Random Hyperparameter search model for RandomForestClassifier
rs_rf.fit(X_train, y_train)

In [ ]:
# Find the best parameter
rs_rf.best_params_

In [ ]:
# Evaluate the randomized Search Random Forest Classifier Model
rs_rf.score(X_test,y_test)

## Hyperparameter Tuning with GridSearchCV

Since our Logistic Regression model provides the best scores so far, we'll try and improve them again using GridSearchCV...

In [ ]:
# Different hyperparameters for our LogisticRegression Model
log_reg_grid = {"C": np.logspace(-4,4,30),
                "solver": ["liblinear"]}

# Setup grid hyperparameter search for Logistic Regresion
gs_log_reg = GridSearchCV(LogisticRegression(),
                          param_grid= log_reg_grid,
                          cv=5,
                          verbose=True)

# Fit grid hyperparameter search model
gs_log_reg.fit(X_train, y_train);

In [ ]:
# Check the best hyperparameters
gs_log_reg.best_params_

In [ ]:
# Evaluate the grid search Logistic Regression model
gs_log_reg.score(X_test, y_test)

## Evaluating our tuned machine learning classifier, beyond accuracy

* ROC curve and AUC score
* Confusion Matrix
* Classification Report
* Precision
* Recall
* F1-score

...and it would be great if cross-validation was used where possible.

To make comparisons and evaluate our trained model, first we need to make predictions.


In [ ]:
# Make predictions with tuned model
y_preds = gs_log_reg.predict(X_test)
y_preds

In [ ]:
y_test.values

In [ ]:
from sklearn.metrics import plot_roc_curve
plot_roc_curve(gs_log_reg,X_test,y_test);

In [ ]:
# ROC Curve
y_probs = gs_log_reg.predict_proba(X_test)
y_probs_positive = y_probs[:,1]
fpr, tpr, thresholds = roc_curve(y_test, y_probs_positive)

def plot_roc(fpr,tpr):
    plt.plot(fpr,tpr,color="blue",label="ROC")
    plt.xlabel("False positive rate (fpr)")
    plt.ylabel("True positive rate (tpr)")
    plt.title("Receiver Operating Characteristics (ROC) Curve")
    plt.legend()
plot_roc(fpr,tpr)   

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,y_probs_positive)

**Confusion Matrix**

In [ ]:
conf_mat = confusion_matrix(y_test, y_preds)
conf_mat

In [ ]:
sns.set(font_scale=1.5)

def plot_conf_mat(conf_mat):
    fig, ax = plt.subplots(figsize=(3,3))
    ax = sns.heatmap(conf_mat,
                     annot=True,
                     cbar=False)
    plt.xlabel("True label")
    plt.ylabel("Predicted label")
#     bottom,top = ax.get_ylim()
#     ax.set_ylim(bottom+0.5,top-0.5)
    
plot_conf_mat(conf_mat)

Now we've got a ROC curve, an AUC metric and a confusion matrix, let's get a classification report as well as cross-validated precision, recall and f1-score.

In [ ]:
print(classification_report(y_test,y_preds))

## Calculate evaluation metrics using cross validation

We're going to calculate accuracy, precision, recall and f1-score of our model using cross-validation and to do so we;ll be using `cross_val_score`


In [ ]:
# Check best hyperparameters

gs_log_reg.best_params_

In [ ]:
# Create a new classifier with best parameters
clf = LogisticRegression(C= 0.20433597178569418,
                         solver= 'liblinear')

In [ ]:
# Cross-validated accuracy:
cv_acc = cross_val_score(clf,
                         X,
                         y,
                         cv=5,
                         scoring="accuracy")
np.mean(cv_acc)

In [ ]:
# Cross-validated precision:
cv_precision = cross_val_score(clf,
                               X,
                               y,
                               cv=5,
                               scoring="precision")
np.mean(cv_precision)

In [ ]:
# Cross-validated recall:
cv_recall = cross_val_score(clf,
                            X,
                            y,
                            cv=5,
                            scoring="recall")
np.mean(cv_recall)

In [ ]:
# Cross-validated f1:
cv_f1 = cross_val_score(clf,
                        X,
                        y,
                        cv=5,
                        scoring="f1")
np.mean(cv_f1)

In [ ]:
# Visualize cross-validated metrics

cv_metrics = pd.DataFrame({"Accuracy": np.mean(cv_acc),
                           "Precision": np.mean(cv_precision),
                           "Recall": np.mean(cv_recall),
                           "F1": np.mean(cv_f1)},
                           index=[0])

In [ ]:
cv_metrics

In [ ]:
cv_metrics.T.plot.barh(title="Cross-validated classification matrix",legend=False);

**Feature Importance**

It is another way of asking which features contributed most to the outcomes of the model and how did they contribute?

Finding feature importance is different for each machine learning model. One way to find feature importance is to search for (MODEL NAME) feature importance.

Let's find the feature importance for our Logistic Regression Model.

In [ ]:
# Fit an instance of LogisticRegressison
gs_log_reg.best_params_

clf = LogisticRegression(C= 0.20433597178569418, solver= 'liblinear')

clf.fit(X_train, y_train);

In [ ]:
# Check coef_
clf.coef_

In [ ]:
df.columns

In [ ]:
clf.coef_[0]

In [ ]:
# Match coef's of features to columns
feature_dict = dict(zip(df.columns,clf.coef_[0]))
feature_dict

In [ ]:
# Visualize feature importance
feature_df = pd.DataFrame(feature_dict,index=[0])

In [ ]:
feature_df

In [ ]:
feature_df.T.plot.barh(title="Feature Importance", legend=False);

In [ ]:
pd.crosstab(df.sex,df.target)

In [ ]:
pd.crosstab(df.slope,df.target)

## 6. Experimentation

If you haven't hit your evaluation metric yet...ask yourself...

* Could you collect more data?
* Could you try a better model? Like CatBoost or XGBoost?
* Could we improve the current models? (beyond what we've done so far)
* If your model is good enough (you have hit your evaluation metric) how would you export it and share it with others?

In [ ]:
from joblib import dump,load

dump(gs_log_reg,filename="best-model.joblib")

In [ ]:
best_model = load("best-model.joblib")
best_model.score(X_test,y_test)